In [1]:
import sys
import os
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt  # graphic library, for plots
#import numba as nb
import filedialogs

In [16]:
import numpy as np
import bokeh.sampledata as sampledata
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from ipywidgets import interact
from bokeh.io import push_notebook, show
from bokeh.models import Range1d
from bokeh.transform import jitter
from bokeh.models import ColumnDataSource

In [3]:
output_notebook() 

Loading BokehJS ...

In [4]:
import pandas

### Importing Data

In [5]:
left_trace = pandas.read_csv('fish_rbc_uv_test2_63x_500Hz_Left.csv')
right_trace = pandas.read_csv('fish_rbc_uv_test2_63x_500Hz_Right.csv')

In [6]:
left_trace = left_trace.rename(columns={"Slice": "frame", "Mean": "left"})
right_trace = right_trace.rename(columns={"Slice": "frame", "Mean": "right"})

In [7]:
df = left_trace.merge(right_trace, left_on='frame', right_on='frame')

In [8]:
df = pandas.concat([left_trace, right_trace['right']], axis=1)

In [24]:
p = figure(width=950, height=600, title="Average RBC OFF events", match_aspect=False, toolbar_location='above')
p.line(df.frame, df.left, color="red", alpha=0.4, line_width=1, legend_label="425 to 435 nm (Hb)")
p.line(df.frame, df.right, color="blue", alpha=0.4, line_width=1, legend_label="405 to 425 nm (HbO2)")
p.yaxis.axis_label = 'Avg. Eventcount in ROI'
p.xaxis.axis_label = 'Time (ms)'
show(p)

In [10]:
df.left = df.left/df.left.mean()
df.right = df.right/df.right.mean()

In [11]:
p = figure(width=950, height=400, title="", tools="", match_aspect=False)
p.line(df.frame, df.left, color="red", alpha=0.4, line_width=1)
#df_nat['mov_avg'] = df_nat['new_cases'].rolling(7).sum()

p.line(df.frame, df.left.rolling(7).sum()/7, color="blue", alpha=0.4, line_width=1)
show(p)

In [12]:
# very handcrafted smoothing
left_above_thresh = np.logical_or(df.left.rolling(1).sum()/1 > 1.5, df.left.rolling(9).sum()/9 > 1)

In [13]:
p = figure(width=1000, height=400, title="", match_aspect=False,  toolbar_location="below")
p.line(df.frame, df.left, color="red", alpha=0.4, line_width=1)
p.line(df.frame, df.right, color="blue", alpha=0.4, line_width=1)
p.line(df.frame, 10*left_above_thresh, color="green", alpha=0.8, line_width=1)
#p.x_range = Range1d(1100, 1300)
#p.x_range = Range1d(1600, 1800)
show(p)

In [14]:
result_df = pandas.DataFrame(columns=['type','left_count', 'right_count','ratio'])
# loop through all timepoints:
rbc_number = 0
left_count = 0
right_count = 0
for i in range(len(left_above_thresh)-1):
    if left_above_thresh[i]==True:
        # add current signal levels to the counts
        left_count = left_count + df.left[i]
        right_count = right_count + df.right[i]
    if left_above_thresh[i]==True and left_above_thresh[i+1] == False:
        # calculate intensity ratio and append to the results
        ratio = left_count/right_count
        # one should not use append in this way, but here we go:
        # adding rows to resulting dataframe
        result_df = result_df._append({'type':'RBC','left_count': left_count, 'right_count': right_count, 'ratio' : ratio}, ignore_index=True)
        # make everything ready for the next rbc
        rbc_number += 1
        left_count = 0
        right_count = 0

/tmp/ipykernel_45477/1811437369.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = result_df._append({'type':'RBC','left_count': left_count, 'right_count': right_count, 'ratio' : ratio}, ignore_index=True)


In [15]:
result_df

,type,left_count,right_count,ratio
0,RBC,151.640337,111.325248,1.362138
1,RBC,235.499497,240.229145,0.980312
2,RBC,153.060472,126.409964,1.210826
3,RBC,77.547930,71.570026,1.083525
4,RBC,207.036087,149.683245,1.383161
5,RBC,288.417969,270.735465,1.065313
6,RBC,175.480419,145.792856,1.203628
7,RBC,173.981937,173.664135,1.001830
8,RBC,187.676632,171.683106,1.093157
9,RBC,213.962777,219.891496,0.973038


In [17]:
result_df_source = ColumnDataSource(result_df)

In [23]:
types = ['RBC']
p = figure(width=200, height=400, title="RBC Intensity Ratios", x_range=types, match_aspect=False, toolbar_location='right')
p.scatter(x=jitter('type',0.3,range=p.x_range),y='ratio', color="red", source=result_df_source)
p.y_range = Range1d(0, 1.5)
p.yaxis.axis_label = '430/410 nm ratio (Putative Hb/HbO2)'
show(p)